In [ ]:
user = "postgres"
password = "postgres"
port = "5432"
database = "nataly"



In [ ]:
jdbc_url = f"jdbc:postgresql://host.docker.internal:{port}/{database}"
connection_properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, lit, coalesce, count, desc, asc, explode, concat_ws

In [ ]:
spark = SparkSession.builder\
        .master("local[1]")\
        .config("spark.hadoop.fs.default.name", f"hdfs://localhost:9000")\
        .config("spark.hadoop.fs.defaultFS", f"hdfs://localhost:9000")\
        .config("spark.jars.packages","org.postgresql:postgresql:42.7.7")\
        .appName('PySpark_Articles')\
        .getOrCreate()

In [ ]:
data = spark.read.json("hdfs://namenode:9000/articles.json")

In [ ]:
data.printSchema()

root
 |-- canonical_url: string (nullable = true)
 |-- collection_id: long (nullable = true)
 |-- comments_count: long (nullable = true)
 |-- cover_image: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- crossposted_at: string (nullable = true)
 |-- description: string (nullable = true)
 |-- edited_at: string (nullable = true)
 |-- flare_tag: struct (nullable = true)
 |    |-- bg_color_hex: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- text_color_hex: string (nullable = true)
 |-- id: long (nullable = true)
 |-- language: string (nullable = true)
 |-- last_comment_at: string (nullable = true)
 |-- organization: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- profile_image: string (nullable = true)
 |    |-- profile_image_90: string (nullable = true)
 |    |-- slug: string (nullable = true)
 |    |-- username: string (nullable = true)
 |-- path: string (nullable = true)
 |-- positive_reactions_count: long (

In [ ]:
articles_raw = data.select('type_of', 'id', 'title', 'description', 'slug', 'url', 'comments_count', 
            'public_reactions_count', 'published_timestamp', 'language', 'subforem_id', 
            'positive_reactions_count', 'created_at', 'published_at', 'reading_time_minutes', 
            'tag_list', 'user.user_id', 'user.name', 'user.username'). \
        withColumn("published_at", col("published_at").cast('timestamp')). \
        withColumn("created_at", col("created_at").cast('timestamp'))

In [ ]:
articles_raw.show(1, False)

+-------+-------+---------------------------+----------------------------------------------------------------------------------------------+-------------------------------+-----------------------------------------------------+--------------+----------------------+--------------------+--------+-----------+------------------------+-------------------+-------------------+--------------------+--------------------------------+-------+-------------------+--------+
|type_of|id     |title                      |description                                                                                   |slug                           |url                                                  |comments_count|public_reactions_count|published_timestamp |language|subforem_id|positive_reactions_count|created_at         |published_at       |reading_time_minutes|tag_list                        |user_id|name               |username|
+-------+-------+---------------------------+-----------------------------

In [ ]:
articles = data.select('type_of', 'id', 'title', 'slug', 'url', 'comments_count', 
            'public_reactions_count', 'published_timestamp', 
            'positive_reactions_count', 'created_at', 'published_at', 'reading_time_minutes', 
            'tag_list', 'user.user_id'). \
        withColumn("published_at", col("published_at").cast('timestamp')). \
        withColumn("created_at", col("created_at").cast('timestamp'))

In [ ]:
articles.show(3)

+-------+-------+--------------------+--------------------+--------------------+--------------+----------------------+--------------------+------------------------+-------------------+-------------------+--------------------+--------------------+-------+
|type_of|     id|               title|                slug|                 url|comments_count|public_reactions_count| published_timestamp|positive_reactions_count|         created_at|       published_at|reading_time_minutes|            tag_list|user_id|
+-------+-------+--------------------+--------------------+--------------------+--------------+----------------------+--------------------+------------------------+-------------------+-------------------+--------------------+--------------------+-------+
|article|2623367|The Second Raku C...|the-second-raku-c...|https://dev.to/li...|             0|                     7|2025-06-25T10:04:03Z|                       7|2025-06-25 09:46:47|2025-06-25 10:04:03|                   5|[rakulang,

In [ ]:
#проверка на дубликаты articles
articles.distinct().count()

2000

In [ ]:
#удаление дубликатов
articles = articles.dropDuplicates()

In [ ]:
users = data.select('user.user_id', 'user.name', 'user.username', 'user.website_url')

In [ ]:
users.show()

+-------+--------------------+--------------------+--------------------+
|user_id|                name|            username|         website_url|
+-------+--------------------+--------------------+--------------------+
| 175135| Elizabeth Mattijsen|              lizmat|https://mastodon....|
| 233697|Ingo Steinke, web...|         ingosteinke|https://www.ingo-...|
| 345658|             BekahHW|             bekahhw|https://bekahhw.com/|
| 950976|      Anmol Baranwal|       anmolbaranwal|https://anmolbara...|
|      3|        dev.to staff|     thepracticaldev|      https://dev.to|
|2971337|       James Bachini|        jamesbachini|https://jamesbach...|
|  55651|     Giorgi Kobaidze|      georgekobaidze|                NULL|
|3284032|   Raghavendra Reddy|raghavendra_reddy...|                NULL|
|  35023|           YCM Jason|            ycmjason|http://www.ycmjas...|
|   9688|       Aurélie Vache|        aurelievache|   http://scraly.com|
| 345658|             BekahHW|             bekahhw|

In [ ]:
#проверка на дубликаты
users.distinct().count() == users.count()

False

In [ ]:
users = users.dropDuplicates()

In [ ]:
users.count()

851

In [ ]:
data_videos = spark.read.json("hdfs://namenode:9000/videos.json")

In [ ]:
data_videos.printSchema()

root
 |-- cloudinary_video_url: string (nullable = true)
 |-- id: long (nullable = true)
 |-- path: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type_of: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |-- user_id: long (nullable = true)
 |-- video_duration_in_minutes: string (nullable = true)
 |-- video_source_url: string (nullable = true)



In [ ]:
data_videos.show()

+--------------------+-------+--------------------+--------------------+-------------+-------------------+-------+-------------------------+--------------------+
|cloudinary_video_url|     id|                path|               title|      type_of|               user|user_id|video_duration_in_minutes|    video_source_url|
+--------------------+-------+--------------------+--------------------+-------------+-------------------+-------+-------------------------+--------------------+
|https://media2.de...|2594229|/kingsmen732/ambi...|Ambient lighting ...|video_article|      {B Mithilesh}|1631667|                    01:19|https://dw71fyauz...|
|https://media2.de...|2597387|/kingsmen732/bypa...|Bypass Netflix’s ...|video_article|      {B Mithilesh}|1631667|                    00:38|https://dw71fyauz...|
|https://media2.de...|2562728|/insightworks/bus...|Business Central ...|video_article|    {Insight Works}|3081293|                    10:39|https://dw71fyauz...|
|https://media2.de...|258892

In [ ]:
videos = data_videos.select('id', 'title', 'type_of', 'user.name', 'user_id', 'video_duration_in_minutes', 'video_source_url')

In [ ]:
videos.show()

+-------+--------------------+-------------+-----------------+-------+-------------------------+--------------------+
|     id|               title|      type_of|             name|user_id|video_duration_in_minutes|    video_source_url|
+-------+--------------------+-------------+-----------------+-------+-------------------------+--------------------+
|2594229|Ambient lighting ...|video_article|      B Mithilesh|1631667|                    01:19|https://dw71fyauz...|
|2597387|Bypass Netflix’s ...|video_article|      B Mithilesh|1631667|                    00:38|https://dw71fyauz...|
|2562728|Business Central ...|video_article|    Insight Works|3081293|                    10:39|https://dw71fyauz...|
|2588924|"Hello, world" 🖥...|video_article|           AI AGI|3185505|                    02:13|https://dw71fyauz...|
|2588138|Project of the We...|video_article|     Riyana Patel|2915485|                    02:00|https://dw71fyauz...|
|2579437|Are you preparing...|video_article|    Mohit Dec

In [ ]:
#проверка на дубликаты
videos.count()

1711

In [ ]:
videos.distinct().count()

1711

In [ ]:
#Записываем в БД


In [ ]:
jdbc_url = "jdbc:postgresql://host.docker.internal:5432/nataly"
connection_properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}

In [ ]:
articles_raw.write.jdbc(
    url=jdbc_url,
    table="pp_articles.articles_raw",
    mode="overwrite", # или "append", "ignore", "error"
    properties=connection_properties
)

In [ ]:
articles.write.jdbc(
    url=jdbc_url,
    table="pp_articles.articles",
    mode="overwrite", # или "append", "ignore", "error"
    properties=connection_properties
)

In [ ]:
users.write.jdbc(
    url=jdbc_url,
    table="pp_articles.users",
    mode="overwrite", # или "append", "ignore", "error"
    properties=connection_properties
)

In [ ]:
videos.write.jdbc(
    url=jdbc_url,
    table="pp_articles.videos",
    mode="overwrite", # или "append", "ignore", "error"
    properties=connection_properties
)

In [ ]:
#замена нулевых значений на ноль в колонке title
#users_with_videos = users_with_videos.withColumn('title', coalesce('title', lit(0)))

In [ ]:
#новый датафрейм с кол-вом видео по пользователям
users_with_videos = videos.groupBy('user_id').agg(count("title").alias("video_cnt"))

In [ ]:
users_with_videos.show(5)

+-------+---------+
|user_id|video_cnt|
+-------+---------+
| 776627|        1|
|1182761|        1|
| 178146|        1|
|1410108|        1|
| 118274|        1|
+-------+---------+
only showing top 5 rows



In [ ]:
#новый датафрейм с кол-вом статей по пользователям 
articles_by_users = articles.groupBy('user_id').agg(count("id").alias("articles_cnt"))

In [ ]:
articles_by_users.show(5)

+-------+------------+
|user_id|articles_cnt|
+-------+------------+
| 127422|           3|
|  69086|           1|
|1406708|           1|
| 385896|           1|
|   6401|           5|
+-------+------------+
only showing top 5 rows



In [ ]:
#джойним два этих датафрейма
top_users = articles_by_users.join(users_with_videos, on='user_id', how='left')

In [ ]:
top_users.show(5)

+-------+------------+---------+
|user_id|articles_cnt|video_cnt|
+-------+------------+---------+
| 127422|           3|     NULL|
|  69086|           1|     NULL|
|1406708|           1|     NULL|
| 385896|           1|     NULL|
|   6401|           5|     NULL|
+-------+------------+---------+
only showing top 5 rows



In [ ]:
top_users.count()

851

In [ ]:
#пользователи с видео
top_users.filter(col('video_cnt').isNotNull()).count()

42

In [ ]:
top_users.filter(col('video_cnt').isNotNull()).show()

+-------+------------+---------+
|user_id|articles_cnt|video_cnt|
+-------+------------+---------+
| 364684|           2|        1|
|1106033|           1|        2|
| 192035|           1|        1|
|  11207|           2|        1|
|    264|          79|        2|
|1262818|           2|        1|
|1438636|           1|        1|
| 555587|           1|       18|
|1031984|           3|        5|
|1054351|           4|        1|
|  40335|           1|        3|
| 345658|          37|        5|
|  28276|           1|        1|
|      1|         104|        1|
|  19970|           3|       31|
| 492757|           9|        1|
|2966215|           1|        2|
| 108648|           1|        2|
|  91775|           1|        1|
| 868253|           1|        1|
+-------+------------+---------+
only showing top 20 rows



In [ ]:
#замена нулевых значений на ноль в колонке video_cnt
top_users = top_users.withColumn('video_cnt', coalesce('video_cnt', lit(0)))

In [ ]:
top_users = top_users.join(users, on='user_id', how='left').select('user_id', 'username', 'articles_cnt', 'video_cnt')

In [ ]:
#лидеры по кол-ву статей
top_users_by_articles = top_users.sort(desc("articles_cnt"))
top_users_by_articles.show()

+-------+------------------+------------+---------+
|user_id|          username|articles_cnt|video_cnt|
+-------+------------------+------------+---------+
|      3|   thepracticaldev|         238|        7|
|      1|               ben|         104|        1|
|    264|              jess|          79|        2|
|  38578|michaeltharrington|          53|        0|
|  31047|             sloan|          52|        0|
| 345658|           bekahhw|          37|        5|
| 968077|       rachelfazio|          25|        0|
|1195604|        anitaolsen|          20|        0|
|   9597|      nickytonline|          15|        9|
|   8745|      jarvisscript|          12|        0|
| 201004|   eevajonnapanula|          11|        0|
|1007771|            erinao|          11|        0|
| 825443|           fmerian|          10|        0|
|  21408|       link2twenty|          10|        0|
|  18254|         thomasbnt|          10|        0|
| 233697|       ingosteinke|          10|        0|
| 397557|   

In [ ]:
#записываем в БД
top_users_by_articles.write.jdbc(
    url=jdbc_url,
    table="pp_articles.top_users_by_articles",
    mode="overwrite", # или "append", "ignore", "error"
    properties=connection_properties
)


In [ ]:
#лидеры по кол-ву видео
top_users_by_videos = top_users.sort(desc("video_cnt"))
top_users_by_videos.show()

+-------+-----------------+------------+---------+
|user_id|         username|articles_cnt|video_cnt|
+-------+-----------------+------------+---------+
|1255335|jguerrero-voxel51|           2|       57|
| 397557|      mishmanners|          10|       32|
|  19970|        bdougieyo|           3|       31|
| 407879|           whykay|           2|       29|
| 555587|          kgilpin|           1|       18|
|   9597|     nickytonline|          15|        9|
|1014611|         proflead|           2|        7|
|      3|  thepracticaldev|         238|        7|
|2915485|      riyanapatel|           1|        6|
|  86540|       cheetah100|           1|        6|
|1031984|   nathan_tarbert|           3|        5|
| 345658|          bekahhw|          37|        5|
|2649629|       d2d_weizhi|           1|        5|
|  48220|    jeremycmorgan|           4|        5|
| 628027|      ssukhpinder|           2|        4|
|  40335|          coreyja|           1|        3|
|1106033|  elanatframework|    

In [ ]:
#записываем в БД
top_users_by_videos.write.jdbc(
    url=jdbc_url,
    table="pp_articles.top_users_by_videos",
    mode="overwrite", # или "append", "ignore", "error"
    properties=connection_properties
)

In [ ]:
users.count()

851

In [ ]:
#отсортируем articles по кол-ву позитивных реакций (>= 1000)
articles.sort(desc("positive_reactions_count")).filter(col('positive_reactions_count') >= 1000).count()

3

In [ ]:
#отсортируем articles по кол-ву позитивных реакций (топ-1000)
top_1000_articles = articles.sort(desc("positive_reactions_count")).limit(1000)

In [ ]:
top_1000_articles.show(3)

+-------+-------+--------------------+--------------------+--------------------+--------------+----------------------+--------------------+------------------------+-------------------+-------------------+--------------------+--------------------+-------+
|type_of|     id|               title|                slug|                 url|comments_count|public_reactions_count| published_timestamp|positive_reactions_count|         created_at|       published_at|reading_time_minutes|            tag_list|user_id|
+-------+-------+--------------------+--------------------+--------------------+--------------+----------------------+--------------------+------------------------+-------------------+-------------------+--------------------+--------------------+-------+
|article|1891624|Say no to console...|say-no-to-console...|https://dev.to/wo...|           121|                  1652|2024-06-17T18:34:53Z|                    1652|2024-06-17 18:34:54|2024-06-17 18:34:53|                   2|        [j

In [ ]:
#разворачиваем список тегов
explode_tags = top_1000_articles.select('id', 'title', 'positive_reactions_count', 'user_id', explode('tag_list').alias('tag'))
explode_tags.show(3)

+-------+--------------------+------------------------+-------+-----------+
|     id|               title|positive_reactions_count|user_id|        tag|
+-------+--------------------+------------------------+-------+-----------+
|1891624|Say no to console...|                    1652|1562696| javascript|
|2052034|11 Practical Ways...|                    1488| 950976|programming|
|2052034|11 Practical Ways...|                    1488| 950976|  beginners|
+-------+--------------------+------------------------+-------+-----------+
only showing top 3 rows



In [ ]:
#самые популярные теги
top_tags = explode_tags.groupBy('tag').agg((count("id")).alias('tag_cnt'))
top_tags.show(3)


+-----------+-------+
|        tag|tag_cnt|
+-----------+-------+
| javascript|    138|
|programming|    169|
|  beginners|    112|
+-----------+-------+
only showing top 3 rows



In [ ]:
top_tags_20 = top_tags.sort(desc('tag_cnt')).limit(20)
top_tags_20.show()

+------------+-------+
|         tag|tag_cnt|
+------------+-------+
|      webdev|    285|
|     discuss|    194|
| programming|    169|
|          ai|    166|
|  javascript|    138|
|devchallenge|    127|
|   beginners|    112|
|  opensource|     94|
| watercooler|     94|
|      career|     91|
|productivity|     73|
|       jokes|     68|
|        top7|     55|
|    learning|     49|
|         css|     49|
|    tutorial|     46|
|     welcome|     33|
|        news|     30|
|     wecoded|     30|
|       react|     28|
+------------+-------+



In [ ]:
#записываем в БД
top_tags_20.write.jdbc(
    url=jdbc_url,
    table="pp_articles.top_tags_20",
    mode="overwrite", # или "append", "ignore", "error"
    properties=connection_properties
)


In [ ]:
top_tags.sort(desc('tag_cnt')).count()

433

In [ ]:
#самые популярные пары тегов
#self-join таблицы explode_tags
explode_tags.createOrReplaceTempView("explode_tags")
popular_tags = spark.sql("""SELECT a.tag as tag_1, b.tag as tag_2
                       FROM explode_tags a, explode_tags b
                       WHERE a.id = b.id and a.tag != b.tag""")

In [ ]:
popular_tags.show()

+---------------+---------------+
|          tag_1|          tag_2|
+---------------+---------------+
|    programming|   productivity|
|    programming|     opensource|
|    programming|      beginners|
|      beginners|   productivity|
|      beginners|     opensource|
|      beginners|    programming|
|     opensource|   productivity|
|     opensource|      beginners|
|     opensource|    programming|
|   productivity|     opensource|
|   productivity|      beginners|
|   productivity|    programming|
|            llm|     opensource|
|            llm|         python|
|            llm|machinelearning|
|machinelearning|     opensource|
|machinelearning|         python|
|machinelearning|            llm|
|         python|     opensource|
|         python|machinelearning|
+---------------+---------------+
only showing top 20 rows



In [ ]:
popular_tags = popular_tags.groupBy('tag_1', 'tag_2').count().sort(desc('count'))

In [ ]:
popular_tags.show()

+------------+------------+-----+
|       tag_1|       tag_2|count|
+------------+------------+-----+
|      webdev|  javascript|  100|
|  javascript|      webdev|  100|
|      webdev| programming|   84|
| programming|      webdev|   84|
| watercooler|     discuss|   80|
|     discuss| watercooler|   80|
| watercooler|       jokes|   66|
|       jokes| watercooler|   66|
|       jokes|     discuss|   65|
|     discuss|       jokes|   65|
|      webdev|   beginners|   45|
|   beginners|      webdev|   45|
|      webdev|devchallenge|   44|
|devchallenge|      webdev|   44|
| programming|  javascript|   41|
|  javascript| programming|   41|
|          ai|      webdev|   37|
|      webdev|          ai|   37|
| programming|   beginners|   37|
|   beginners| programming|   37|
+------------+------------+-----+
only showing top 20 rows



In [ ]:
popular_tags = popular_tags.drop_duplicates(['count']).sort(desc('count'))

In [ ]:
popular_tags = popular_tags.withColumn('tags_pair', concat_ws(' - ', col('tag_1'), col('tag_2')))

In [ ]:
popular_tags_pair = popular_tags.select('tags_pair', 'count').limit(40)
popular_tags_pair.show()

+--------------------+-----+
|           tags_pair|count|
+--------------------+-----+
| webdev - javascript|  100|
|webdev - programming|   84|
|watercooler - dis...|   80|
| watercooler - jokes|   66|
|     discuss - jokes|   65|
|  webdev - beginners|   45|
|devchallenge - we...|   44|
|programming - jav...|   41|
|programming - beg...|   37|
|   devchallenge - ai|   36|
|    ai - programming|   33|
| webdev - opensource|   27|
|devchallenge - fr...|   26|
|devchallenge - ja...|   25|
|programming - pro...|   24|
|      react - webdev|   23|
|     career - webdev|   22|
|    javascript - css|   21|
|   webdev - tutorial|   20|
|  javascript - react|   18|
+--------------------+-----+
only showing top 20 rows



In [ ]:
#записываем в БД
popular_tags_pair.write.jdbc(
    url=jdbc_url,
    table="pp_articles.popular_tags_pair",
    mode="overwrite", # или "append", "ignore", "error"
    properties=connection_properties
)


In [ ]:
#самые популярные статьи по реакциям
articles.select('user_id', 'title', 'comments_count', 'public_reactions_count', 'reading_time_minutes', 'tag_list').sort(desc('public_reactions_count')).limit(50).show()

+-------+--------------------+--------------+----------------------+--------------------+--------------------+
|user_id|               title|comments_count|public_reactions_count|reading_time_minutes|            tag_list|
+-------+--------------------+--------------+----------------------+--------------------+--------------------+
|1562696|Say no to console...|           121|                  1652|                   2|        [javascript]|
| 950976|11 Practical Ways...|           107|                  1488|                  11|[programming, beg...|
| 862981|Using the Ollama ...|             8|                  1018|                   2|[llm, machinelear...|
|1169909|SSL for localhost...|            63|                   988|                   4|[webdev, nginx, s...|
|1408863|Mastering Relatio...|            50|                   749|                  20|[webdev, learning...|
|    264|Join us for the n...|            48|                   702|                   6|[devchallenge, fr...|
|

In [ ]:
#самые комментируемые статьи 
articles.select('user_id', 'title', 'comments_count', 'public_reactions_count', 'reading_time_minutes', 'tag_list').sort(desc('comments_count')).limit(50).show()

+-------+--------------------+--------------+----------------------+--------------------+--------------------+
|user_id|               title|comments_count|public_reactions_count|reading_time_minutes|            tag_list|
+-------+--------------------+--------------+----------------------+--------------------+--------------------+
|  31047|Welcome Thread - ...|           354|                    47|                   1|           [welcome]|
| 262904|Do You Still Use ...|           320|                   216|                   1|[discuss, git, te...|
|  31047|Welcome Thread - ...|           306|                    49|                   1|           [welcome]|
|  31047|Welcome Thread - ...|           287|                    50|                   1|           [welcome]|
|  31047|Welcome Thread - ...|           282|                    34|                   1|           [welcome]|
|  31047|Welcome Thread - ...|           280|                    37|                   1|           [welcome]|
|